In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from model.model import SimpleMLP
from data_handling.dataset import EmbeddingsToAgeDataset

In [5]:
INPUT_DIM = 1026 # 1024 features + bmi + height
HIDDEN_DIM = 512
OUTPUT_DIM = 1
model = SimpleMLP(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)

model.load_state_dict(torch.load("model/simple_mlp.pth"))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/tmp/ipykernel_165132/713862183.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model/simple_mlp.pth"))


SimpleMLP(
  (fc1): Linear(in_features=1026, out_features=512, bias=True)
  (hidden_fc_1): Linear(in_features=512, out_features=512, bias=True)
  (hidden_fc_2): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (batch_norm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [6]:
BATCHSIZE = 32
val_loader = torch.utils.data.DataLoader(
        EmbeddingsToAgeDataset("../data/healthy_val.csv"),
        batch_size=BATCHSIZE,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
        drop_last=True
    )

In [ ]:

def get_accuracy_and_loss(model, val_loader):
    criterion = torch.nn.MSELoss()
    total_loss = 0
    total_samples = 0
    correct_predictions = 0

    with torch.no_grad():
        for batch in val_loader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs).squeeze()
            targets = targets.squeeze()

            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            total_samples += inputs.size(0)
            correct_predictions += (torch.abs(outputs - targets) < 5).sum().item()

    accuracy = correct_predictions / total_samples
    average_loss = total_loss / total_samples

    return accuracy, average_loss

acc, loss = get_accuracy_and_loss(model, val_loader)
print(f"Validation Accuracy: {acc:.4f}")
print(f"Validation Loss: {loss:.4f}")

Validation Accuracy: 0.4761
Validation Loss: 55.9537
